In [8]:
!pip -q install --upgrade google-auth-oauthlib google-api-python-client pandas
import importlib.metadata as im, glob, os
print("google-auth-oauthlib =", im.version("google-auth-oauthlib"))
print("google-api-python-client =", im.version("google-api-python-client"))
print("client_secret present:", bool(glob.glob("client_secret.json")))


google-auth-oauthlib = 1.2.2
google-api-python-client = 2.181.0
client_secret present: True


In [9]:
from urllib.parse import urlparse, parse_qs
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
import os, json

SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]
TOKEN_PATH = "token.json"

def get_youtube_via_loopback_copy(scopes=SCOPES, token_path=TOKEN_PATH):
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, scopes)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes)
            flow.redirect_uri = "http://localhost:8080/"  # 关键：显式设置 redirect_uri
            auth_url, _ = flow.authorization_url(
                access_type="offline",
                include_granted_scopes="true",
                prompt="consent",
            )
            print("在浏览器打开并完成授权：\n", auth_url)
            redirect_response = input("\n授权完成后，把【浏览器地址栏的完整重定向URL】粘贴到此处：\n").strip()
            try:
                code = parse_qs(urlparse(redirect_response).query)["code"][0]
            except Exception:
                code = redirect_response  # 允许只粘 code
            flow.fetch_token(code=code)
            creds = flow.credentials
        with open(token_path, "w") as f:
            f.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def get_my_channel_id_safe(yt):
    """安全获取频道ID，避免 KeyError，并在失败时给出清晰提示。"""
    try:
        resp = yt.channels().list(part="id", mine=True, fields="items(id)").execute()
    except HttpError as e:
        print("YouTube API error:", e)
        return None
    items = resp.get("items", [])
    if not items:
        print("⚠️  当前账号没有 YouTube 频道或无权限。请先在网页端创建频道，然后删除 token.json 重新授权。")
        return None
    return items[0]["id"]

yt = get_youtube_via_loopback_copy()
my_channel_id = get_my_channel_id_safe(yt)
print("My channel id:", my_channel_id)


My channel id: UCMj60L5aE0ZsIi8fmroy_fg


In [11]:
import pandas as pd

def list_my_subscriptions(yt, max_pages=None):
    rows, page, pages = [], None, 0
    while True:
        resp = yt.subscriptions().list(
            part="snippet",
            mine=True, maxResults=50, pageToken=page,
            fields="nextPageToken,items(snippet(title,publishedAt,resourceId/channelId))"
        ).execute()
        for it in resp.get("items", []):
            sn = it["snippet"]
            rows.append({
                "ChannelTitle": sn["title"],
                "ChannelId": sn["resourceId"]["channelId"],
                "SubscribedAt": sn.get("publishedAt"),
            })
        page = resp.get("nextPageToken"); pages += 1
        if not page or (max_pages and pages >= max_pages): break
    return pd.DataFrame(rows)

def list_my_liked_videos(yt, max_pages=None):
    rows, page, pages = [], None, 0
    while True:
        resp = yt.videos().list(
            part="id,snippet,contentDetails,statistics",
            myRating="like", maxResults=50, pageToken=page,
            fields=("nextPageToken,items("
                    "id,"
                    "snippet(title,channelTitle,publishedAt),"
                    "contentDetails(duration),"
                    "statistics(viewCount,likeCount))")
        ).execute()
        for v in resp.get("items", []):
            rows.append({
                "VideoId": v["id"],
                "Title": v["snippet"]["title"],
                "ChannelTitle": v["snippet"]["channelTitle"],
                "PublishedAt": v["snippet"]["publishedAt"],
                "Duration": v["contentDetails"]["duration"],
                "ViewCount": v["statistics"].get("viewCount"),
                "LikeCount": v["statistics"].get("likeCount"),
            })
        page = resp.get("nextPageToken"); pages += 1
        if not page or (max_pages and pages >= max_pages): break
    return pd.DataFrame(rows)

# —— 实际调用 ——
if my_channel_id:
    df_subs = list_my_subscriptions(yt)
    display(df_subs.head(10)); print("Total subscriptions:", len(df_subs))
    df_likes = list_my_liked_videos(yt)
    display(df_likes.head(10)); print("Total liked videos:", len(df_likes))

    # 可选导出
    df_subs.to_csv("my_subscriptions.csv", index=False)
    df_likes.to_csv("my_liked_videos.csv", index=False)
else:
    print("未获取到频道 ID，先按提示创建频道并重新授权。")


,ChannelTitle,ChannelId,SubscribedAt
0,怪咖 Outcasts,UCVpZkO-pOOr6z4NdxAKegPg,2025-09-11T03:08:25.051076Z
1,拜托了小翔哥,UCBPMSrO-Ljh_jGeFX5HDmkQ,2025-09-11T03:07:57.501044Z
2,美食挖掘机,UCoHcW0XwwRaMlVea5C03geA,2025-09-11T02:08:13.85328Z


Total subscriptions: 3


,VideoId,Title,ChannelTitle,PublishedAt,Duration,ViewCount,LikeCount
0,HzkbELvR4Hc,尋找最好吃的炸鷄！吃爆10間炸雞連鎖店！| Finding the Best Fried C...,怪咖 Outcasts,2025-08-15T10:00:35Z,PT27M21S,155653,6848
1,qtYt88z_tdw,在日本，吃牛肉飯代表你很窮？實際探店帶你看日本的窮人餐廳都是哪些，沒錢又怎麼在日本生活呢？,冷水TV・日本生活與日本文化,2025-06-10T10:01:31Z,PT12M22S,1621627,27392
2,6sMA1QNerNI,从3毛到100元都能吃到什么样的泡面,拜托了小翔哥,2025-09-02T10:00:10Z,PT14M14S,218432,3349
3,XGbVp62P64k,全球最魔性，印度航空！天上吃什么？,觅食Meetfood,2024-12-22T01:00:21Z,PT8M22S,575688,9608
4,wZhOsKw10yA,来东北吃盒饭啦！12一份6个菜！居然还送水！ #街头美食 #路边摊美味,美食挖掘机,2025-01-23T11:43:00Z,PT58S,760065,30086
5,6iqIwXjNyH0,修杰楷跟著千千爆吃捷運美食！逼出小鳥胃極限，一路吃到投降？！【修TIME】Ep23 | ...,修TIME,2025-09-10T06:44:41Z,PT27M18S,266866,4533


Total liked videos: 6
